In [ ]:
!pip install kaggle


In [ ]:
! pip install tensorflow-gpu


In [ ]:

import pandas as pd


In [ ]:
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt


In [ ]:

df_fake=pd.read_csv('/kaggle/input/fake-and-real-news-dataset/Fake.csv')
df_true=pd.read_csv('/kaggle/input/fake-and-real-news-dataset/True.csv')

In [ ]:
df_fake.head()

In [ ]:
df_true['truth'] = 1
df_fake['truth'] = 0
df = pd.concat([df_true,df_fake], axis=0, ignore_index=True)
df.shape

In [ ]:
df.head()

In [ ]:
df.iloc[1]["title"]

In [ ]:
! pip install Unidecode
! pip install wordninja

In [ ]:
import re
import unidecode
import wordninja
import nltk
import string

In [ ]:
# nltk.download()

In [ ]:
def clean_text(text):
  # lower-case all characters
    text=text.lower()
    
    # remove twitter handles
    text= re.sub(r'@\S+', '',text) 
    
    # remove urls
    text= re.sub(r'http\S+', '',text) 
    text= re.sub(r'pic.\S+', '',text)
      
    # replace unidecode characters
    text=unidecode.unidecode(text)
      
    # regex only keeps characters
    text= re.sub(r"[^a-zA-Z+']", ' ',text)
    
    # keep words with length>1 only
    text=re.sub(r'\s+[a-zA-Z]\s+', ' ', text+' ') 
    text="".join([i for i in text if i not in string.punctuation])
    words=nltk.tokenize.word_tokenize(text)
    stopwords = nltk.corpus.stopwords.words('english')
    text =" ".join([i for i in words if i not in stopwords and len(i)>2])
    

    
    



    

#     split words like 'whatisthis' to 'what is this'
    def preprocess_wordninja(sentence):      
        def split_words(x):
            x=wordninja.split(x)
            x= [word for word in x if len(word)>1]
            return x
        new_sentence=[ ' '.join(split_words(word)) for word in sentence.split() ]
        return ' '.join(new_sentence)
    
    text=preprocess_wordninja(text)
 
    # regex removes repeated spaces, strip removes leading and trailing spaces
    text= re.sub("\s[\s]+", " ",text).strip()  
    
    
    return text


In [ ]:
clean_text("hello world is not jskakkz qkn  klncnask tttt")

In [ ]:
df["title"]=df["title"].apply(clean_text)

In [ ]:
df.iloc[1]["title"]

In [ ]:
from sklearn.model_selection import train_test_split

In [ ]:
X = df['title']
y=df["truth"]

In [ ]:
display(X.head())

In [ ]:
y.value_counts(normalize=True)

In [ ]:
 X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=42)

In [ ]:
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow.keras.layers import Dense, Input, GlobalMaxPooling1D
from tensorflow.keras.layers import Conv1D, MaxPooling1D, Embedding,LSTM
from tensorflow.keras.models import Model

In [ ]:
# Convert sentences to sequences
MAX_VOCAB_SIZE = 20000
tokenizer = Tokenizer(num_words=MAX_VOCAB_SIZE)
tokenizer.fit_on_texts(X_train)
sequences_train = tokenizer.texts_to_sequences(X_train)
sequences_test = tokenizer.texts_to_sequences(X_test)

In [ ]:
# get word -> integer mapping
word2idx = tokenizer.word_index
V = len(word2idx)
print('Found %s unique tokens.' % V)

In [ ]:
# pad sequences so that we get a N x T matrix
data_train = pad_sequences(sequences_train)
print('Shape of data train tensor:', data_train.shape)

# get sequence length
T = data_train.shape[1]

In [ ]:
data_test = pad_sequences(sequences_test, maxlen=T)
print('Shape of data test tensor:', data_test.shape)

In [ ]:
# Create the model

# We get to choose embedding dimensionality
D = 20

# Hidden state dimensionality
M = 15

# Note: we actually want to the size of the embedding to (V + 1) x D,
# because the first index starts from 1 and not 0.
# Thus, if the final index of the embedding matrix is V,
# then it actually must have size V + 1.

i = Input(shape=(T,))
x = Embedding(V + 1, D)(i)
x = LSTM(M, return_sequences=True)(x)
x = GlobalMaxPooling1D()(x)
x = Dense(1, activation='sigmoid')(x)

model = Model(i, x)


In [ ]:

model.compile(
  loss='binary_crossentropy',
  optimizer='adam',
  metrics=['accuracy']
)


print('Training model...')
r = model.fit(
  data_train,
  y_train,
  epochs=5,
  validation_data=(data_test, y_test)
)

In [ ]:
plt.plot(r.history['loss'], label='loss')
plt.plot(r.history['val_loss'], label='val_loss')
plt.legend()

In [ ]:
plt.plot(r.history['accuracy'], label='acc')
plt.plot(r.history['val_accuracy'], label='val_acc')
plt.legend()

In [ ]:
model.evaluate(data_train, y_train)

In [ ]:
model.evaluate(data_test,y_test)

In [ ]:
predictions= model.predict(data_test)

In [ ]:
predictions[:10]

In [ ]:

predict=[]
for i in predictions:
    if i<0.5:
        predict.append(0)
    else:
        predict.append(1)
predict[:10]

In [ ]:
from sklearn.metrics import classification_report

print(classification_report(y_test, predict))